In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW>

In [3]:
from time import time
from preprocessing.Preprocessor import Preprocessor

In [4]:
pp = Preprocessor(spark)

In [5]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from metrics.metrics import Metrics

encoding_lst = [pp.to_bow]
#pp.to_tfidf --> add to list 

for i in encoding_lst:
    train = i('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = True)
    test =  i('Cleaned Data/test.csv')
    #train = pp.balance_classes(train)
    train.cache()
    test.cache()
    nb = NaiveBayes(smoothing = 1.0, modelType="multinomial")
    model = nb.fit(train)
    test_m = Metrics()
    test = model.transform(test).select('prediction', 'label')
    #y = str(i)
    #x = str((test_m.f1(test)))
    #print(y + x, end = "")
    test_m.f1(test)
    test_m.confusion_matrix(test)

F1 score for given DataFrame: 72.1

                Predictions
            negative    neutral     positive    
negative    2525        378         1228        
neutral     438         320         721         
positive    955         856         8972        
(True labels are on the side)

